Question 

We have to find the location of the theft! We are given a dataset of images (cats and dogs). The test dataset consists of 2 images which we need to decode. We have 4 locations (00, 01, 10 & 11) and the theft will occur in one of these places. Use a CNN model to analyse the dataset and feed the test data to recieve your final answer.

In [ ]:
# Importing necessary modules.

import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import datasets, layers, models
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Dense , Flatten , Dropout
import cv2
import imghdr

In [ ]:
# Directory containing the dataset.

data_dir = "/kaggle/input/final-dataset/dataset_1"

In [ ]:
# Only images of these format should be present. Everything else like .gif etc will be removed.

image_exts = ['jpeg' , 'jpg' , 'bmp' , 'png']

for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir , image_class)):
        image_path = os.path.join(data_dir , image_class , image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print("Image not in ext list {}".format(image_path))
                os.remove(image_path)
        except Exception as e:
            print("Issue with image {}".format(image_path))

In [ ]:
# Storing all the images inside the directory to the variable "data".

data = tf.keras.utils.image_dataset_from_directory('/kaggle/input/final-dataset/dataset_1')

In [ ]:
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()
data_iterator , batch[0].shape

In [ ]:
fig , ax = plt.subplots(ncols = 4 , figsize = (20,20))
for idx , img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# Here, Dog -> 1 & Cat -> 0.

In [ ]:
scaled = batch[0]/255
data = data.map(lambda x,y : (x/255 , y))

In [ ]:
# Splitting the dataset into train, validation and test data.

train_size = int(len(data)*.7)
val_size = int(len(data)*.2) + 1
test_size = int(len(data)*.1)
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

**Question 1**

Now that our dataset is well prepared, use a CNN model to learn the entire dataset. Use an appropriate number of hidden layers, flatten and dense functions along with an activation function to get the necessary output! Also, use am optimiser too.

In [ ]:
# Answer

model = Sequential()
model.add(Conv2D(16 , (3,3) , 1 , activation='relu' , input_shape = (256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32 , (3,3) , 1 , activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(32 , (3,3) , 1 , activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(256 , activation='relu'))
model.add(Dense(1 , activation='sigmoid'))

model.compile('adam' , loss = tf.losses.BinaryCrossentropy() , metrics = ['accuracy'])

In [ ]:
logdir = 'Logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

Question

Train the dataset! :D

Use an appropriate number of epochs to get the final CNN model.

In [ ]:
# Answer

hist = model.fit(train , epochs = 20 , validation_data=val , callbacks=[tensorboard_callback])

**Question 3**

Now that your CNN model is ready, utilise the test dataset to get your final answer!

In [ ]:
# Answer

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model

# Function to preprocess and predict for a single image
def preprocess_and_predict(image_path, model):
    img = cv2.imread(image_path)
    resize = tf.image.resize(img, (256, 256))
    yhat = model.predict(np.expand_dims(resize.numpy() / 255.0, 0))[0][0]
    label = 1 if yhat > 0.5 else 0
    return label

# Directory containing the test images
test_data_dir = '/kaggle/input/final-dataset/test'  # Replace with your test images path

# Iterate over all images in the test directory (The one the kids should type)
predictions = []
for img_file in os.listdir(test_data_dir):
    img_path = os.path.join(test_data_dir, img_file)
    label = preprocess_and_predict(img_path, model)
    predictions.append(label)
    print(f"Image: {img_file}, Prediction: {label}")